# Pokemon Classifier
## Imports and constants

In [1]:
import os
from PIL import Image
import numpy as np
from random import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import csv

train_dirs = ['batch1', 'batch2', 'batch3', 'batch4']
dataset_folder = 'dataset'
IMG_SIZE = 128
MODEL_NAME = 'classifier.h5'

2024-04-07 10:36:53.419768: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 10:36:53.464447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 10:36:53.464473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 10:36:53.465592: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 10:36:53.472503: I tensorflow/core/platform/cpu_feature_guar

## Label images

In [2]:
def label_img(img):
    label = img.split('_gen_')[1].split('.')[0]
    if   label == '1': return [1, 0, 0, 0, 0, 0, 0, 0, 0]
    elif label == '2': return [0, 1, 0, 0, 0, 0, 0, 0, 0]
    elif label == '3': return [0, 0, 1, 0, 0, 0, 0, 0, 0]
    elif label == '4': return [0, 0, 0, 1, 0, 0, 0, 0, 0]
    elif label == '5': return [0, 0, 0, 0, 1, 0, 0, 0, 0]
    elif label == '6': return [0, 0, 0, 0, 0, 1, 0, 0, 0]
    elif label == '7': return [0, 0, 0, 0, 0, 0, 1, 0, 0]
    elif label == '8': return [0, 0, 0, 0, 0, 0, 0, 1, 0]
    elif label == '9': return [0, 0, 0, 0, 0, 0, 0, 0, 1]

# Load data

In [3]:
def load_data():
    training_data = []
 
    for directory in train_dirs:
        for img in os.listdir(f'{dataset_folder}/{directory}'):
            if '_gen_' not in img: continue

            label = label_img(img)

            path = os.path.join(f'{dataset_folder}/{directory}', img)
            img = Image.open(path).convert('RGB')

            training_data.append([np.array(img), np.array(label)])

    return training_data

## Classifier architecture

In [4]:
classifier = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.8),
    Dense(9, activation='softmax')
])

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classifier.summary()

2024-04-07 10:36:55.946032: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 14, 128)       0

2024-04-07 10:36:55.995067: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-07 10:36:55.998161: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-07 10:36:56.001516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

## Train the model

In [5]:
data = load_data()

x = np.array([i[0] for i in data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array([i[1] for i in data])

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.25, random_state=0, stratify=y_train)

history = classifier.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val))
classifier.save(MODEL_NAME)

Epoch 1/100


2024-04-07 10:37:12.558716: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-04-07 10:37:12.680540: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-07 10:37:13.188222: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-07 10:37:14.020173: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3818c0aab0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-07 10:37:14.020209: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-04-07 10:37:14.026281: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1712486234.129791    6086 device_compiler.h:186] Compiled

338/338 [==============================] - 11s 18ms/step - loss: 2.4115 - accuracy: 0.1189 - val_loss: 2.1960 - val_accuracy: 0.1119
Epoch 2/100
338/338 [==============================] - 5s 15ms/step - loss: 2.1918 - accuracy: 0.1218 - val_loss: 2.1946 - val_accuracy: 0.1175
Epoch 3/100
338/338 [==============================] - 5s 15ms/step - loss: 2.1741 - accuracy: 0.1406 - val_loss: 2.1427 - val_accuracy: 0.1544
Epoch 4/100
338/338 [==============================] - 5s 15ms/step - loss: 2.1586 - accuracy: 0.1410 - val_loss: 2.0912 - val_accuracy: 0.1678
Epoch 5/100
338/338 [==============================] - 5s 15ms/step - loss: 2.0736 - accuracy: 0.1861 - val_loss: 1.9529 - val_accuracy: 0.2658
Epoch 6/100
338/338 [==============================] - 5s 15ms/step - loss: 1.8435 - accuracy: 0.3103 - val_loss: 1.5427 - val_accuracy: 0.4261
Epoch 7/100
338/338 [==============================] - 5s 15ms/step - loss: 1.4459 - accuracy: 0.4477 - val_loss: 1.2007 - val_accuracy: 0.5222
Epo

/opt/saturncloud/envs/saturn/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Evaluate the model

In [6]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

y_pred = classifier.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average="macro"))
print("Recall:", recall_score(y_test, y_pred, average="macro"))
print("F1-score:", f1_score(y_test, y_pred, average="macro"))

113/113 [==============================] - 1s 4ms/step
Accuracy: 0.9511111111111111
Precision: 0.9516381368336453
Recall: 0.9511111111111111
F1-score: 0.9511987517448964
